In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice
from collections import Counter, defaultdict
from nltk.util import ngrams

In [2]:
df = pd.read_csv('data/to_eval/results/2016_G06F_RESULTS.csv')
df2 = pd.read_csv('data/knowledge_space/cleaned/2016KS_G06F_clean.csv')

In [3]:
list_applications_to_eval = list(df['application_number'])
list_applications_KS = list(df2['application_number'])

In [4]:
print(len(list_applications_to_eval), len(list_applications_KS))

9234 173661


In [5]:
mypath = 'data/2016/2016/'
_2014_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
len(_2014_files)

373255

In [6]:
#pOUR LA ks -- les 5 années avant
mypath_1 = 'data/2015/2015/'
_1_files = [f for f in listdir(mypath_1) if isfile(join(mypath_1, f))]
mypath_2 = 'data/2014/2014/'
_2_files = [f for f in listdir(mypath_2) if isfile(join(mypath_2, f))]
mypath_3 = 'data/2013/2013/'
_3_files = [f for f in listdir(mypath_3) if isfile(join(mypath_3, f))]
mypath_4 = 'data/2012/2012/'
_4_files = [f for f in listdir(mypath_4) if isfile(join(mypath_4, f))]
mypath_5 = 'data/2011/2011/'
_5_files = [f for f in listdir(mypath_5) if isfile(join(mypath_5, f))]

In [7]:
def return_year(file_path):
    #print(file_path)
    if file_path in _1_files:
        return mypath_1
    if file_path in _2_files:
        return mypath_2
    if file_path in _3_files:
        return mypath_3
    if file_path in _4_files:
        return mypath_4
    if file_path in _5_files:
        return mypath_5

In [8]:
def pairing(list_ipc):
    pairs = []
    for L in range(len(list_ipc) + 1):
        for subset in itertools.combinations(list_ipc, L):
            if len(subset) == 2:
                pairs.append(subset)
    return pairs

In [9]:
### On considère une combinaison comme étant une association avec la main_IPC -- le reste ferait pas de sens?
###### POUR LA KS on a juste besoin de toutes les combinaisons
combi_direct_KS = []
combi_all_KS = []
for i in tqdm(range(len(list_applications_KS))):
    patent_path = str(list_applications_KS[i]) + '.json'
    correct_path = return_year(patent_path)
    patent_path = correct_path + patent_path
    with open(patent_path) as f:
        d = json.load(f)
        f.close()

    class_mainIPC = d['main_ipcr_label']
    non_main =  d['ipcr_labels']
    #rint(type(class_mainIPC), type(non_main))
    full_classes = [class_mainIPC] + non_main
    patent_combi = []
    list_full = []
    if len(non_main) != 0:
        for class_ipc in non_main:
            patent_combi.append((class_mainIPC, class_ipc))
        list_full = pairing(full_classes)
    combi_all_KS.extend(list_full)
    combi_direct_KS.extend(patent_combi)

100%|██████████████████████████████████████████████████████████████████████████| 173661/173661 [39:39<00:00, 72.99it/s]


In [10]:
print(len(combi_all_KS), len(combi_direct_KS), combi_all_KS[0:3], combi_direct_KS[0:3])

367198 250148 [('G06F758', 'G06F758'), ('G06F3041', 'G06F3041'), ('G06F3041', 'G06F30346')] [('G06F758', 'G06F758'), ('G06F3041', 'G06F3041'), ('G06F3041', 'G06F30346')]


In [11]:
### On considère une combinaison comme étant une association avec la main_IPC -- le reste ferait pas de sens?
combi_direct = []
combi_all = []
for i in tqdm(range(len(list_applications_to_eval))):
    file = str(list_applications_to_eval[i]) + '.json'
    patent_path = mypath + file
    with open(patent_path) as f:
        d = json.load(f)
        f.close()

    class_mainIPC = d['main_ipcr_label']
    non_main =  d['ipcr_labels']
    full_classes = [class_mainIPC] + non_main
    patent_combi = []
    list_full = []
    if len(full_classes) > 20:
        full_classes = full_classes[0:10]
    if len(non_main) != 0:
        for class_ipc in non_main:
            patent_combi.append((class_mainIPC, class_ipc))
        list_full = pairing(full_classes)
    combi_all.append(list_full)
    combi_direct.append(patent_combi)

100%|█████████████████████████████████████████████████████████████████████████████| 9234/9234 [00:44<00:00, 205.27it/s]


In [12]:
print(len(combi_all), len(combi_direct), combi_all[0:3], combi_direct[0:3])

9234 9234 [[('G06F30481', 'G06F30481')], [('G06F950', 'G06F950'), ('G06F950', 'G06F1208'), ('G06F950', 'G06F9445'), ('G06F950', 'G06F1202'), ('G06F950', 'G06F1208'), ('G06F950', 'G06F9445'), ('G06F950', 'G06F1202'), ('G06F1208', 'G06F9445'), ('G06F1208', 'G06F1202'), ('G06F9445', 'G06F1202')], [('G06F3041', 'G06F3041'), ('G06F3041', 'H01L2732'), ('G06F3041', 'G09G33225'), ('G06F3041', 'H01L2732'), ('G06F3041', 'G09G33225'), ('H01L2732', 'G09G33225')]] [[('G06F30481', 'G06F30481')], [('G06F950', 'G06F950'), ('G06F950', 'G06F1208'), ('G06F950', 'G06F9445'), ('G06F950', 'G06F1202')], [('G06F3041', 'G06F3041'), ('G06F3041', 'H01L2732'), ('G06F3041', 'G09G33225')]]


In [13]:
#Score estimation
score_direct = []
for list_tuple in combi_direct:
    #print(list_tuple)
    norm_value = len(list_tuple)
    score = 0
    for tuple in list_tuple:
        #print(tuple)
        #Just so we don't count pairs of the same class
        if tuple[0] == tuple[1]:
            #print('prout')
            norm_value = norm_value - 1
            continue
        #print('prout2')
        if tuple in combi_direct_KS:
            continue
        else:
            score+=1
    if norm_value == 0:
        score_direct.append(score)
    else:
        #Normalizing the score
        score_direct.append(score/norm_value)

In [14]:
print(len(score_direct), score_direct[0:10])

9234 [0, 0.0, 0.0, 1.0, 0, 0.0, 0.0, 0, 0.0, 0]


In [15]:
#Score estimation
score_all = []
for list_tuple in combi_all:
    #print(list_tuple)
    norm_value = len(list_tuple)
    score = 0
    for tuple in list_tuple:
        #print(tuple)
        #Just so we don't count pairs of the same class
        if tuple[0] == tuple[1]:
            #print('prout')
            norm_value = norm_value - 1
            continue
        #print('prout2')
        if tuple in combi_all_KS:
            continue
        else:
            score+=1
    if norm_value == 0:
        score_all.append(score)
    else:
        #Normalizing the score
        score_all.append(score/norm_value)

In [16]:
print(len(score_all), score_all[0:10])

9234 [0, 0.0, 0.2, 1.0, 0, 0.0, 0.0, 0, 0.0, 0]


In [17]:
df['Novel_IPC_direct'] = score_direct
df['Novel_IPC_all'] = score_all

In [18]:
df.to_csv('data/to_eval/results/2016_G06F_RESULTS_withIPC.csv', index=False)